In [ ]:
import numpy as np
import time as time
import os
import glob
import copy

# Set up matplotlib and use a nicer set of plot parameters
# %config InlineBackend.rc = {}
import matplotlib as mpl
# mpl.rc('mathtext',fontset='stixsans')
# mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook
import scipy.optimize, scipy.ndimage
import tqdm

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

import constants
import lyafxcorr_kg

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
# PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
# SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

PiBound = (min(PiEdges), max(PiEdges) )

In [ ]:
OBS_DIR = os.path.join(constants.XCORR_DIR_BASE, 'obs')

XCorr_3d=np.load(os.path.join(OBS_DIR, f"xcorr_3dhst_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_3d = np.transpose(XCorr_3d)
    
XCorr_zD = np.load(os.path.join(OBS_DIR, f"xcorr_zDeep_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_zD = np.transpose(XCorr_zD)
    
XCorr_mosdef = np.load(os.path.join(OBS_DIR, f"xcorr_mosdef_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_mosdef = np.transpose(XCorr_mosdef)
    
XCorr_vuds = np.load(os.path.join(OBS_DIR, f"xcorr_vuds_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_vuds = np.transpose(XCorr_vuds)

XCorr_clamato = np.load(os.path.join(OBS_DIR, f"xcorr_clamato_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_clamato = np.transpose(XCorr_clamato)

XCorr_all = np.load(os.path.join(OBS_DIR, f"xcorr_all_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_all = np.transpose(XCorr_all)

print(XCorr_all.shape)

xcorr_list = [XCorr_3d, XCorr_zD, XCorr_mosdef, XCorr_vuds, XCorr_clamato]
titles = ['3DHST', 'zDeep', 'MOSDEF', 'VUDS', 'CLAMATO']
redshift_dispersions = [9.79, 3, 1.5, 1.92, 2] # TODO: undo?
#redshift_dispersions = [9.79, 3, 1.5, 1.92, 2.5]
#redshift_dispersions = [9.79, 0, 0.5, 1.92, 0.5] # Blatantly eye-adjusted until the 1D fits look okay.
#redshift_dispersions = [0, 0, 0, 0, 0]

In [ ]:
COVAR_DIR = os.path.join(constants.XCORR_DIR_BASE, 'bootstrap')

Covar_zdeep = np.load(os.path.join(COVAR_DIR, f'covar_zDeep_n21600_{constants.DATA_VERSION}.npy'))
Covar_vuds  = np.load(os.path.join(COVAR_DIR, f'covar_VUDS_n21600_{constants.DATA_VERSION}.npy'))
Covar_clamato = np.load(os.path.join(COVAR_DIR, f'covar_CLAMATO_n21600_{constants.DATA_VERSION}.npy'))
Covar_mosdef = np.load(os.path.join(COVAR_DIR, f'covar_MOSDEF_n21600_{constants.DATA_VERSION}.npy'))
Covar_3dhst = np.load(os.path.join(COVAR_DIR, f'covar_3DHST_n21600_{constants.DATA_VERSION}.npy'))

covar_list = [Covar_3dhst, Covar_zdeep, Covar_mosdef, Covar_vuds, Covar_clamato]

In [ ]:
parallel_bin_centers = []
for i in range(len(PiEdges) - 1):
    lb, ub = PiEdges[i], PiEdges[i+1]
    parallel_bin_centers.append(np.mean([lb, ub]))
parallel_bin_centers = np.array(parallel_bin_centers)

In [ ]:
mask = np.ones_like(parallel_bin_centers)
mask[:5] = 0
mask[-5:] = 0
mask = mask.astype(bool)
print(np.max(parallel_bin_centers[mask]) - np.min(parallel_bin_centers[mask]))

In [ ]:
# Reduce covar in adding.
def reduce_covar(covar):
    parallel_size = XCorr_all.shape[1]
    if covar.ndim != 4:
        if covar.shape[0] == XCorr_all.size:
            covar = covar.reshape(*XCorr_all.shape, *XCorr_all.shape)
        else:
            raise RuntimeError
    # Verified through linearity of covariance when summing that this is the correct procedure.
    return np.sum(covar, axis=(0, 2))
    
reduced_covar_list = []
for covar in covar_list:
    reduced_covar_list.append(reduce_covar(np.copy(covar)))
    assert np.allclose(reduced_covar_list[-1], reduced_covar_list[-1].T)

In [ ]:
parallel_bin_centers / .125

In [ ]:
parallel_bin_centers_even = np.arange(np.min(parallel_bin_centers), np.max(parallel_bin_centers) + .125, .125)
parallel_bin_center_mask = np.logical_or.reduce([parallel_bin_centers_even == i for i in parallel_bin_centers])
assert np.sum(parallel_bin_center_mask) == len(parallel_bin_centers)

def peak(x, *p, smoothing_scale=0):
    if smoothing_scale > 0:
        x = parallel_bin_centers_even
    A, mu, sigma, C = p
    dist = np.minimum(np.abs(x - (mu + sigma)), np.abs(x - (mu - sigma)))
    dist[x > mu + sigma] = 0
    dist[x < mu - sigma] = 0
    if smoothing_scale > 0:
        y_even = A * scipy.ndimage.gaussian_filter(dist**3, sigma=(smoothing_scale / .125)) + C
        return y_even[parallel_bin_center_mask]
    else:
        return A * dist**3 + C
    #return A * scipy.ndimage.gaussian_filter(dist**3, sigma=smoothing_scale) + C
    #return A*np.exp(-(x-mu)**2/(2.*sigma**2)) + C
    
def peak_factory(z_disp):
    return lambda x, *p: peak(x, *p, smoothing_scale=z_disp / 0.75)

In [ ]:
def fit_peak(xcorr, covar, mask, covar_diag, z_disp):
    reduced_xcorr = np.sum(xcorr, axis=0)
    x = parallel_bin_centers[mask]
    reduced_xcorr = reduced_xcorr[mask]
    covar = covar[mask, :][:, mask]
    if covar_diag:
        covar = np.diag(covar)
    background = np.mean(np.concatenate([reduced_xcorr[:3], reduced_xcorr[-3:]]))#np.mean([reduced_xcorr[0], reduced_xcorr[-1]])
    fit_coeff, _ = scipy.optimize.curve_fit(peak_factory(z_disp), x, reduced_xcorr, 
                                            p0=[(np.min(reduced_xcorr) - background) / 30**3, 0, 30, background], 
                                            #bounds=([-np.inf, -30, 10, -np.inf], [0, 30, np.inf, np.inf]),
                                            sigma=covar,
                                            maxfev=10000)
    return fit_coeff

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(11, 6))

for xcorr, covar, t, z_disp, ax in zip(xcorr_list, reduced_covar_list, titles, redshift_dispersions, axes.flatten()):
    reduced_xcorr = np.sum(xcorr, axis=0)
    # covar = np.diag(np.diag(covar))
    ax.errorbar(parallel_bin_centers, reduced_xcorr, color='black', label='Data', lw=1, ls='--')
    yerr = np.diag(covar)**0.5
    ax.fill_between(parallel_bin_centers, reduced_xcorr - yerr, reduced_xcorr + yerr, step='mid', color='grey', alpha=0.2)
    fit_coeff_diagonly = fit_peak(xcorr, covar, np.ones(xcorr.shape[1]).astype(bool), True, z_disp)
    fit_coeff_full = fit_peak(xcorr, covar, np.ones(xcorr.shape[1]).astype(bool), False, z_disp)
    # background = np.mean([reduced_xcorr[0], reduced_xcorr[-1]])
    # fit_coeff, _ = scipy.optimize.curve_fit(peak, parallel_bin_centers, reduced_xcorr, p0=[(np.min(reduced_xcorr) - background) / 30**3, 0, background], sigma=covar)
    # print(fit_coeff_full)
    smoothed_peak = peak_factory(z_disp)
    #ax.plot(parallel_bin_centers, smoothed_peak(parallel_bin_centers, *fit_coeff_diagonly), color='red', label='Diagonal covar only', lw=1)
    ax.plot(parallel_bin_centers, smoothed_peak(parallel_bin_centers, *fit_coeff_full), color='red', label='Full covar', lw=1)
    #ax.set_title(f'{t} diag delta={fit_coeff_diagonly[1]:.3}, full delta={fit_coeff_full[1]:.3}', fontsize=8)
    ax.set_title(f'{t} $\delta_z$={fit_coeff_full[1]:.2f} Mpc/h', fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.tick_params(axis='both', which='minor', labelsize=10)
axes[-1][-1].legend(*axes[0][0].get_legend_handles_labels(), loc='upper left')
axes[-1][-1].axis('off')
xlabel_str = r'$\pi$ [Mpc/h]'
ylabel_str = r'$\sigma$-summed cross-correlation'
axes[0][0].set_ylabel(ylabel_str, fontsize=10)
axes[1][0].set_ylabel(ylabel_str, fontsize=10)
axes[1][0].set_xlabel(xlabel_str, fontsize=10)
axes[1][1].set_xlabel(xlabel_str, fontsize=10)
axes[1][2].set_xlabel(xlabel_str, fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'obs', 'peak_fits.pdf'))

In [ ]:
plt.imshow(reduced_covar_list[-1])

# Mock validation

In [ ]:
def fit_plot_deltaz_mock(survey_name, z_disp, mask=None, covar_diag=True, truth=None):
    N_VOL = 63
    N_ABS = 3
    N_GAL = 10
    if mask is None:
        mask = np.ones_like(parallel_bin_centers).astype(bool)
    deltaz = []
    # covar = reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_{survey_name}_v0.npy')))
    n_fit_fail = 0
    for igal in tqdm.tqdm(range(N_GAL)):
        for iabs in range(N_ABS):
            for ivol in (range(1, N_VOL + 1)):
                abssuffix = '{0:03d}'.format(ivol+iabs*N_VOL)
                galsuffix = '{:03d}'.format(ivol+igal*N_VOL)
                xcorr_data = np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'crosscorr', f'xcorrmock_{survey_name}_g{galsuffix}_a{abssuffix}_v0.npy'))
                
#                 lyafil = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'skewers', 'pixel_radecz_mock_'+abssuffix+'.bin')
#                 lyapix = lyafxcorr_kg.lyapix(lyafil,cosmo=cosmo)
#                 lyapix_skewerrec = lyapix.gen_SkewerRec()
#                 npix = lyapix.npix

#                 galfil = os.path.join(constants.XCORR_DIR_BASE, 'mock', 'gal', 'cat_galmock_nonuniq_'+galsuffix+'.dat')
#                 gal = ascii.read(galfil, format='ipac')
#                 assert gal['ra'].unit == u.degree
#                 assert gal['dec'].unit == u.degree
#                 source_mask = np.array([s.lower() == survey_name.lower() for s in gal['source']])
#                 gal = gal[source_mask]
#                 ngal = len(gal)
#                 Coord = SkyCoord(ra=gal['ra'], dec=gal['dec'], distance=cosmo.comoving_distance(gal['zspec']))
                
#                 def do_resampling(n_sub_partial, seed=None):
#                     XCorrSamples_Partial = np.zeros([len(SigEdges)-1, len(PiEdges)-1, n_sub_partial])
#                     rng = np.random.default_rng(seed)
#                     for ii in range(n_sub_partial):
#                         # Make a copy of the pixels and resample
#                         LyaPixTmp = copy.deepcopy(lyapix)
#                         LyaPixTmp.rng = rng
#                         LyaPixTmp = LyaPixTmp.resample_skewer(SkewerRec=lyapix_skewerrec)
#                         # Resample galaxy positions
#                         GalCoordTmp = Coord[rng.choice(ngal,ngal,replace=True)]
#                         XCorrTmp, _ = lyafxcorr_kg.xcorr_gal_lya(GalCoordTmp, LyaPixTmp,SigEdges, PiEdges,
#                                                           cosmo=cosmo,verbose=0)
#                         XCorrSamples_Partial[:, :, ii] = XCorrTmp
#                     return XCorrSamples_Partial
                
#                 XCorrSamples = do_resampling(100)
#                 boot_covar = np.cov(XCorrSamples)
                boot_covar = np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_{survey_name}_v0.npy'))
                boot_covar = reduce_covar(boot_covar)
                try:
                    deltaz.append(fit_peak(xcorr_data, boot_covar, mask, covar_diag, z_disp)[1])
                except RuntimeError:
                    n_fit_fail += 1
                    continue
    plt.hist(deltaz, bins=20)
    print(f'Number of failed fits: {n_fit_fail}')
    print(f'{np.mean(deltaz)} +- {np.std(deltaz)}')
    plt.title(survey_name)
    plt.xlabel(r'Fitted $\delta_z$ [Mpc/h]')
    if truth is not None:
        plt.axvline(truth, color='black', label=r'True $\delta_z$')
        plt.legend()
    plt.show()

In [ ]:
def average_xcorrs(survey_name):
    xcorr_path_list = glob.glob(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'crosscorr', f"xcorrmock_{survey_name}_*_{constants.DATA_VERSION}.npy"))
    xcorr_plot_sum = np.zeros_like(XCorr_all)
    for i, p in enumerate(xcorr_path_list):
        xcorr_plot_sum += np.load(p)
    return xcorr_plot_sum / len(xcorr_path_list)

Avg_XCorr_mosdef = average_xcorrs('mosdef')
Avg_XCorr_vuds = average_xcorrs('vuds')
Avg_XCorr_zD = average_xcorrs('zDeep')
Avg_XCorr_3d = average_xcorrs('3dhst')
Avg_XCorr_clamato = average_xcorrs('clamato')

In [ ]:
COVAR_DIAG = False

In [ ]:
fit_plot_deltaz_mock('mosdef', redshift_dispersions[2], mask=None, covar_diag=COVAR_DIAG, truth=0.87)
fit_peak(Avg_XCorr_mosdef, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_mosdef_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      COVAR_DIAG, redshift_dispersions[2])[1]

In [ ]:
fit_plot_deltaz_mock('zDeep', redshift_dispersions[1], mask=None, covar_diag=COVAR_DIAG, truth=2.77)
fit_peak(Avg_XCorr_zD, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_zDeep_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      COVAR_DIAG, redshift_dispersions[1])[1]

In [ ]:
fit_plot_deltaz_mock('vuds', redshift_dispersions[3], mask=None, covar_diag=COVAR_DIAG, truth=2.65)
fit_peak(Avg_XCorr_vuds, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_vuds_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      COVAR_DIAG, redshift_dispersions[3])[1]

In [ ]:
fit_plot_deltaz_mock('clamato', redshift_dispersions[4], mask=None, covar_diag=COVAR_DIAG, truth=1.48)
fit_peak(Avg_XCorr_clamato, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_clamato_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      COVAR_DIAG, redshift_dispersions[4])[1]

In [ ]:
fit_plot_deltaz_mock('3dhst', redshift_dispersions[0], mask=None, covar_diag=COVAR_DIAG, truth=1)
fit_peak(Avg_XCorr_3d, 
                      reduce_covar(np.load(os.path.join(constants.XCORR_DIR_BASE, 'mock', 'covar', f'covar_rawmock1890_3dhst_v0.npy'))), 
                      np.ones_like(parallel_bin_centers).astype(bool), 
                      COVAR_DIAG, redshift_dispersions[0])[1]